
part of speech model.ipynb<br>
Automatically generated by Colaboratory.<br>
Original file is located at<br>
    https://colab.research.google.com/drive/1wntZTVBpkuhU7hy60nzog1HbKo6U-71Q<br>
#Imports<br>


In [ ]:
import torch
import json
import re
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
BATCH_SIZE=64


#Data Preparation<br>
##Tokinizer<br>
###Tokinizer Util<br>


In [ ]:
class TokinizerDatasetUtils():
  def __init__(self, maxDictLen=10000):
    self.maxDictLen=maxDictLen
    self.tokinizerDict=dict({"UNK":0, "PAD":1})

In [ ]:
  def loadTokinizerDictionary(self, filePath)->None:
    jsonfile=open(filePath)
    jsonObject=json.load(jsonfile)
    self.tokinizerDict=jsonObject

In [ ]:
  def createDataset(self, dataset: list[list[str]], fileSavePath=None)->None:
    for sentance in dataset:
      for word in sentance:
        if self.maxDictLen==len(self.tokinizerDict):
          break
        if word not in self.tokinizerDict:
          self.tokinizerDict[word]=len(self.tokinizerDict)
    if fileSavePath !=None:
      self._saveDictionary(fileSavePath)

In [ ]:
  def extractSentanceText(self, fileSavePath=None, *DatasetfilePaths)->None:
    dataset=self._extractData("sentence", DatasetfilePaths)
    self.createDataset(dataset, fileSavePath)

In [ ]:
  def extractLabelText(self, fileSavePath=None, *DatasetfilePaths)->None:
    dataset=self._extractData("labels", DatasetfilePaths)
    self.createDataset(dataset, fileSavePath)
    self.createDataset(dataset, fileSavePath)

In [ ]:
  def _extractData(self, field: str, DatasetfilePaths):
    dataset=[]
    for DatasetfilePath in DatasetfilePaths:
      jsonFile = open(DatasetfilePath)
      jsonArray = json.load(jsonFile)

In [ ]:
      for object in jsonArray:
        dataset.append(object[field])
    return dataset

In [ ]:
  def _saveDictionary(self, savePath):
    with open(savePath, "w") as outfile:
      json.dump(self.tokinizerDict, outfile)


###Tokinizer Class
<br>
class Tokinizer(TokinizerDatasetUtils):<br>
  def __init__(self, maxDictLen=10000, maxSquenceLength=100):<br>
    super().__init__(maxDictLen)<br>
    self.maxSquenceLength=maxSquenceLength<br>
  def encode(self, sentanceList: list[str])->list[int]:<br>
    encodedSentance=[]<br>
    for word in sentanceList:<br>
      if len(encodedSentance)>=self.maxSquenceLength:<br>
        break<br>
      if word in self.tokinizerDict.keys():<br>
        encodedSentance.append(self.tokinizerDict[word])<br>
      else:<br>
        encodedSentance.append(self.tokinizerDict["UNK"])<br>
    return self.addPaddingToEncoding(encodedSentance)<br>
  def addPaddingToEncoding(self, encoding: list[int])->list[int]:<br>
    paddingLengthRequired=self.maxSquenceLength-len(encoding)<br>
    paddingArray=[self.tokinizerDict["PAD"]]*paddingLengthRequired<br>
    return encoding+paddingArray<br>
  def decode(self, encodedSentance: list[int])->list[str]:<br>
    decodedString=""<br>
    dictKeys=list(self.tokinizerDict.keys())<br>
    for token in encodedSentance:<br>
      try:<br>
        wordPosition=list(self.tokinizerDict.values()).index(token)<br>
        decodedString+=dictKeys[wordPosition]<br>
      except ValueError:<br>
        raise ValueError(f"token {token} was not found in dictionary")<br>
    return decodedString<br>
  def __len__(self):<br>
    return len(self.tokinizerDict)<br>
#Make Dataset


In [ ]:
class PosDataset(Dataset):

In [ ]:
  def __init__(self,filePaths: list):
    self.tokinizerSentance=Tokinizer(maxDictLen=2000)
    self.tokinizerLabel=Tokinizer(maxDictLen=2000)
    self.tokinizerSentance.extractSentanceText("tokinizerSentanceDict.json", "train.json", "test.json")
    self.tokinizerSentance.loadTokinizerDictionary("tokinizerSentanceDict.json")
    self.tokinizerLabel.extractSentanceText("tokinizerLabelDict.json", "train.json", "test.json")
    self.tokinizerLabel.loadTokinizerDictionary("tokinizerLabelDict.json")

In [ ]:
    self.filePaths=filePaths
    self.datasetSentances=torch.tensor(self.unpackSentances())
    self.datasetLabels=torch.tensor(self.unpackLabels())

In [ ]:
  def unpackSentances(self)->list[int]:
    sentanceList=[]
    for filePath in self.filePaths:
      jsonFile = open(filePath)
      jsonArray = json.load(jsonFile)

In [ ]:
      for object in jsonArray:
        tokinizedSentace=self.tokinizedSentace(object["sentence"])
        sentanceList.append(tokinizedSentace)
    return sentanceList

In [ ]:
  def unpackLabels(self)->list[int]:
    labelsList=[]
    for filePath in self.filePaths:
      jsonFile = open(filePath)
      jsonArray = json.load(jsonFile)

In [ ]:
      for object in jsonArray:
        tokinizedLabels=self.tokinizedLabels(object["labels"])
        labelsList.append(tokinizedLabels)
    return labelsList

In [ ]:
  def tokinizedSentace(self, sentance: list[str])->list[int]:
    return self.tokinizerSentance.encode(sentance)

In [ ]:
  def tokinizedLabels(self, labels: list[str])->list[int]:
    return self.tokinizerLabel.encode(labels)

In [ ]:
  def __getitem__(self,idx):
    return self.datasetSentances[idx],self.datasetLabels[idx]

In [ ]:
  def __len__(self)->int:
    return len(self.datasetSentances);

In [ ]:
dataset=PosDataset(["test.json","test.json"])


##Dataloader
<br>
class Dataloader():<br>
  def __init__(self, dataset, trainSize: float):<br>
    self.dataset=dataset<br>
    self.trainSize=trainSize<br>
  def trainTestDataloader(self):<br>
    trainDataset, testDataset=self.splitDataset(self.dataset)<br>
    trainDataloader=DataLoader(trainDataset,batch_size=BATCH_SIZE,shuffle=True)<br>
    testDataloader=DataLoader(testDataset,batch_size=BATCH_SIZE,shuffle=True)<br>
    return trainDataloader, testDataloader<br>
  def splitDataset(self, dataset):<br>
    trainSize = int(self.trainSize * len(self.dataset))<br>
    testSize = len(self.dataset) - trainSize<br>
    trainDataset, testDataset = torch.utils.data.random_split(dataset, [trainSize, testSize])<br>
    return trainDataset, testDataset<br>
#Model


In [ ]:
class PosModel(nn.Module):
  def __init__(self, vocabilarySize, hiddenSize, output):
    super().__init__()
    self.squential=nn.Sequential(
      nn.Embedding(vocabilarySize, hiddenSize),
      nn.RNN(hiddenSize, output, 10)
    )

In [ ]:
  def forward(self, sentance: torch.tensor):
    return self.squential(sentance)

In [ ]:
inputSize=len(dataset.tokinizerSentance.tokinizerDict)
outputSize=len(dataset.tokinizerLabel.tokinizerDict)

In [ ]:
posModel=PosModel(inputSize,5,outputSize).to(device)


#Optimizer and loss
<br>
def parametersCount(model):<br>
  return sum(p.numel() for p in model.parameters() if p.requires_grad)<br>
print(f"model parameters: {parametersCount(posModel):,}")<br>
optimizer=torch.optim.Adam(posModel.parameters(),lr=0.001)<br>
loss=nn.CrossEntropyLoss()<br>
Training


In [ ]:
class ModelTrainer():
  def __init__(self, model, optimizer, loss, dataset, epochs, device):
    self.model=model
    self.optimizer=optimizer
    self.loss=loss
    self.epochs=epochs
    self.device=device
    self.trainDataloader, self.testDataloader = Dataloader(dataset,0.8).trainTestDataloader()
    self.startTraining()

In [ ]:
  def startTraining(self)->None:
    for epoch in tqdm(range(self.epochs)):
      trainGenerator=self.train()
      trainLoss, trainAccuracy=self.unpackGenerator(trainGenerator)
      
      testGenerator=self.test()
      testLoss, testAccuracy=self.unpackGenerator(testGenerator)
      print(f"epoch {epoch} | train loss: {trainLoss:.2f}, train accuracy: {trainAccuracy:.2f} | test loss: {testLoss:.2f}, test accuracy: {testAccuracy:.2f}")

In [ ]:
  def train(self):
    for input, target in self.trainDataloader:
      self.optimizer.zero_grad()
      prediction=self.model(input)
      loss, accuracy=self.getLossAndAccuracy(prediction, target)
      loss.requires_grad = True
      loss.backward()
      self.optimizer.step()
      yield loss, accuracy

In [ ]:
  def test(self):
    for input, target in self.testDataloader:
      self.optimizer.zero_grad()
      prediction=self.model(input)

In [ ]:
      loss, accuracy=self.getLossAndAccuracy(prediction, target)
      yield loss, accuracy

In [ ]:
  def unpackGenerator(self, generator):
    generator=next(iter(generator))
    loss, accuracy=generator[0],generator[1]
    return loss, accuracy

In [ ]:
  def getLossAndAccuracy(self,prediction,target):
    prediction=prediction[0].argmax(2).float()
    target=target.float()
    prediction_loss=self.loss(prediction,target)
    prediction_acc=self.accuracy(prediction,target)
    return prediction_loss,prediction_acc

In [ ]:
  def accuracy(self,predictions,targets):
    assert predictions.shape == targets.shape, "Shapes of predictions and targets must match."
    num_correct = (predictions == targets).sum().item()
    total_samples = targets.numel()
    accuracy_value = num_correct / total_samples
    return accuracy_value*100

In [ ]:
ModelTrainer(posModel, optimizer, loss, dataset, 100, device)